# Model

Wright-Fisher pop-dyn model with a fitness landscape defined as the "empirical avearge" of many fitness landscapes.
Each individual has a different, partial empirical experience of the landcscape (e.g. it samples only a subset of resources in the time of a division). This makes the genotype-to-phenotype map stochastic. Precisely, we assume this stochasticity is given by independent individual mini-batching at each generation. 

Consider a large set of species $S\gg 1$ and define a set of $M$ fitness landscapes $\vec f^\mu = (f^\mu_1, \dots, f^\mu_S)$, $\mu=1,\dots, M$.

Null model: $\vec f^0= \frac{1}{M}\sum_\mu \vec f^\mu$. 

Algorithm:

0. initialize population (start from all population concentrated on a single genotype for comparison to SGD)
1. batching (select B landscapes for each individual)
2. compute fitness
3. multinomial sampling of the new population (WF proliferation)
4. mutation
5. define inheritance of batches
6. repeat 1-4


The goal is to compare how the stochastic version of the model performs compared to the null model. Precisely: When does exploiting the individual variability help or harm **generalization?**

Generalization performance:
$$\chi_g=\sum_a z_a(t) \mathbb E_{\vec h_a^{M+1}}\left[f_a^{M+1}\right]$$


### Random interface model
Species: $\vec s_a\in\{0,1\}^L$ 

Fields: $\vec h^\mu = \vec h^0 + \vec {\delta h}^\mu$, where $h^0=(1,\dots,1,0,\dots, 0)$

$$f^\mu_a=\frac{\vec h^\mu \cdot \vec s_a}{\lVert \vec s_a\rVert}$$



In [ ]:
push!(LOAD_PATH,"./model_interface")
using Distributed
addprocs(4)
@everywhere using SWF

######### simulation hyperparameters #########
# state space hyperparameters (hypercube - binary sequence)
L = 16   # sequence length
K = 4   # number of fixed field sites
M = 100    # number of landscapes
p_env = ones(M)./M;  # probability of each landscape
nu = 0.001;
Delta = 4.;
pk = 0.2;
F0 = 1.; #plays no role in interface model
gen_new = false;

# mini-batching hyperparameters
B = 1   # batch size
p_inherit = 0.;   # probability of inheriting the same landscape

# simulation hyperparameters
nsteps = 10000   # number of simulation steps
save_every = 50 # save data every save_every steps
save_after = 0   # save data only after save_after steps
ncopies = 100  # number of copies of the simulation

function main_wrapper(set::Settings,evo::EvoSettings,init_cond::String)
    pop_ini, pop_ref_ini = initialize_pop(set,init_cond);   
    println("initialized")
    nsteps,save_every,save_after,ncopies = evo.nsteps,evo.save_every,evo.save_after,evo.ncopies
    @distributed for n in 1:evo.ncopies 
        main_evolve(n,set,pop_ini,pop_ref_ini,nsteps,save_every,save_after,init_cond) 
    end
    return
end;


if gen_new
    #new landscapes
    consmatrix, fieldmatrix = generate_fields(M,L,K,pk,Delta,F0);
else
    #use old landscapes
    consmatrix,fieldmatrix = copymatrix("prova_interface/data/Delta$(Delta)_F0$(F0)_nu$(nu)_N128_M$(M)_B1_L$(L)/half_fixed/long_K$(K)_pk$(pk)/")
    fieldmatrix = convert(Matrix{Float64},fieldmatrix)
    consmatrix = reshape(convert(Vector{Float64},consmatrix),(1,L))
end
maxc = maximum(consmatrix);
 
for N in 2 .^range(10,11)
    set = Settings(Delta,F0,nu,N,M,B,p_inherit,L,K,pk,p_env,consmatrix,fieldmatrix,maxc);
    evoset = EvoSettings(nsteps,save_every,save_after,ncopies)
    
    #init:zeros
    @time main_wrapper(set,evoset,"zeros")
    println("zeros finish")
    
    #init:ones
    @time  main_wrapper(set,evoset,"ones")
    println("ones finish")
    
    #init:rand
    for rand_ini in 1:5
        init_cond = "rand$(rand_ini)"
        @time  main_wrapper(set,evoset,init_cond)
        println("rand $(rand_ini) finish")
    end
end

initialized
  0.035196 seconds (68.14 k allocations: 4.181 MiB, 95.74% compilation time)
zeros finish
initialized
  0.043194 seconds (54.76 k allocations: 3.303 MiB, 96.59% compilation time)
ones finish
initialized
  0.001345 seconds (27.05 k allocations: 1.940 MiB)
rand 1 finish
initialized
  0.001828 seconds (27.05 k allocations: 1.939 MiB)
rand 2 finish
initialized
  0.001385 seconds (27.05 k allocations: 1.939 MiB)
rand 3 finish
initialized
  0.002742 seconds (27.05 k allocations: 1.939 MiB)
rand 4 finish
initialized
  0.001396 seconds (27.05 k allocations: 1.939 MiB)
rand 5 finish
initialized
  0.002873 seconds (50.31 k allocations: 3.556 MiB)
zeros finish
initialized
  0.002404 seconds (51.63 k allocations: 3.650 MiB)
ones finish
initialized
  0.002427 seconds (51.63 k allocations: 3.652 MiB)
rand 1 finish
initialized
  0.002923 seconds (51.62 k allocations: 3.650 MiB)
rand 2 finish
initialized
  0.003780 seconds (51.62 k allocations: 3.650 MiB)
rand 3 finish
initialized
  0.0030